In [1]:
%pip install selenium openpyxl pandas 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 셀레니움 초기 설정1 (selenium 모듈, 관련 옵션 설정)
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options #
from selenium.webdriver.common.by import By
import time
from datetime import datetime
import csv

# 셀레니움 초기 설정2
chrome_options = Options()
chrome_options.add_argument("--headless")  # 헤드리스 모드로 실행, 
chrome_options.add_argument("--start-maximized")  # 창 크기 최대화된 상태로 크롬 브라우저를 실행
chrome_options.add_experimental_option("detach", True) # 스크립트 실행 끝나도 브라우저 창 닫지 않음. 

driver = webdriver.Chrome(options=chrome_options) # 크롬 드라이버를 사용해 브라우저 제어.

# 검색어 및 URL 설정
AI = "Artificial Intelligence"
base_url = "https://edition.cnn.com/search?q={}&from={}&size=10&page={}&sort=newest&types=all&section=" 

# CSV 파일 생성
csv_file = "[NEW]CNN_news_with_content.csv" # 해당 파일인 csv_file을 생성
with open(csv_file, "w", newline="", encoding="utf-8-sig") as file: 
    writer = csv.writer(file) # 데이터를 쓰기 위한 객체인 writer를 초기화
    writer.writerow(["날짜", "기사제목", "기사요약", "기사내용"]) # writer 첫줄에 열 제목(4)을 추가 

    max_page = 33  # 최대 페이지 수(2024년의 기사를 세었을 때 총 33페이지)
    for page in range(1, max_page + 1):
        try:
            # 페이지 URL 구성
            offset = 10 * (page - 1)
            url = base_url.format(AI, offset, page)
            driver.get(url)
            time.sleep(3)

            # 기사 목록 가져오기
            articles = driver.find_elements(By.CSS_SELECTOR, "div.container__text.container_list-images-with-description__text")
            if not articles:
                print(f"Page {page}: No articles found.")
                continue

            for article in articles:
                try:
                    # 기사 제목 및 링크
                    title_element = article.find_element(By.CSS_SELECTOR, "span.container__headline-text")
                    title = title_element.text.strip()
                    link = title_element.find_element(By.XPATH, ".").get_attribute('data-zjs-href')
                    print(link)

                    # 요약
                    try:
                        summary_element = article.find_element(By.CSS_SELECTOR, "div.container__description.container_list-images-with-description__description.inline-placeholder")
                        summary = summary_element.text.strip()
                    except Exception:
                        summary = "요약 없음"

                    # 날짜
                    try:
                        date_element = article.find_element(By.CSS_SELECTOR, "div.container__date.container_list-images-with-description__date.inline-placeholder")
                        raw_date = date_element.text.strip()
                        parsed_date = datetime.strptime(raw_date, "%b %d, %Y")
                        date = parsed_date.strftime("%Y-%m-%d")
                    except Exception:
                        date = "날짜 없음"

                    # 추가: 기사 내용 크롤링
                    content = "내용 없음"
                    try:
                        # 기사 링크로 이동
                        
                        driver.execute_script("window.open('');") #새탭 열기
                        driver.switch_to.window(driver.window_handles[1]) #새탭 전환
                        driver.get(link)
                        time.sleep(3) #페이지 로딩 시간

                        # 기사 내용 확인 및 본문 가져오기
                        selectors = [
                            "p.paragraph.inline-placeholder.vossi-paragraph",  # 기본 기사
                            "div.video-resource__description"  # 동영상 기사 내용
                                    ] #추가한 내용
                        for selector in selectors:
                            paragraphs = driver.find_elements(By.CSS_SELECTOR, selector)
                            if paragraphs:
                                content = "\n".join([p.text for p in paragraphs if p.text.strip()])
                                break
                        
                        driver.close() # 새탭닫기
                        driver.switch_to.window(driver.window_handles[0]) #원 탭 전환
                
                    except Exception as e:
                        content = "Inappropriate Article Content"
                        print(f"Error occurs while searching article: {e}")
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])

                    # CSV 파일에 저장
                    writer.writerow([date, title, summary, content])

                    # 출력
                    print(f"제목: {title}")
                    print(f"날짜: {date}")
                    print(f"요약: {summary}")
                    print(f"내용: {content}\n")

                except Exception as e:
                    print(f"Error parsing article: {e}")

        except Exception as e:
            print(f"Error on page {page}: {e}")

driver.quit()
print(f"Scraping completed. Data saved to {csv_file}.")